In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr

In [2]:
yf.pdr_override()

In [3]:
df = pd.read_csv("stocksymbols - Sheet1.csv")
syms = list(df['Symbol'])
print(syms)

['SBIN.NS', 'AXISBANK.BO', 'IOC.BO', 'NMDC.BO', 'KRITIIND.BO', 'TATAPOWER.NS', 'AUROLAB.BO', 'RESONANCE.BO', 'ANDHRAPET.BO', 'MANALIPETC.BO', 'GAIL.NS', 'SAIL.NS', 'HDFCBANK.BO', 'ONGC.NS', 'INFY.BO', 'BHAGYAPROP.BO', 'FEDERALBNK.NS', 'WIPRO.BO', 'IDEA.NS', 'ALOKTEXT.BO', 'BANKBARODA.BO', 'BHEL.NS', 'DISHTV.BO', 'VEDL.BO', 'PTC.NS', 'PRESSMN.NS', 'VEDL.BO']


In [4]:
start = "2016-04-01"
end = str(int(start[:4]) + 3) + start[4:]
stock_data = pdr.get_data_yahoo(syms[:5], start = start, end = end)['Adj Close']
stock_data.head()

[*********************100%***********************]  5 of 5 completed


,AXISBANK.BO,IOC.BO,KRITIIND.BO,NMDC.BO,SBIN.NS
Date,,,,,
2016-04-01,440.503143,70.593666,32.958870,79.307602,191.382736
2016-04-04,436.142731,72.174149,33.401604,79.712654,190.453461
2016-04-05,423.551331,72.101608,33.352413,79.186081,180.329224
2016-04-06,416.300262,73.744942,32.466949,82.264420,179.840118
2016-04-07,417.868042,73.790176,31.729065,82.588448,177.981552


In [5]:
returns = stock_data.pct_change()
returns.head()

,AXISBANK.BO,IOC.BO,KRITIIND.BO,NMDC.BO,SBIN.NS
Date,,,,,
2016-04-01,NaN,NaN,NaN,NaN,NaN
2016-04-04,-0.009899,0.022388,0.013433,0.005107,-0.004856
2016-04-05,-0.028870,-0.001005,-0.001473,-0.006606,-0.053159
2016-04-06,-0.017120,0.022792,-0.026549,0.038875,-0.002712
2016-04-07,0.003766,0.000613,-0.022727,0.003939,-0.010335


In [6]:
mean_daily_returns = np.array(returns.mean()).reshape(-1, 1)
cov = returns.cov()

In [7]:
mean_daily_returns

array([[0.00092921],
       [0.00191748],
       [0.00041526],
       [0.00045104],
       [0.00090449]])

In [8]:
cov

,AXISBANK.BO,IOC.BO,KRITIIND.BO,NMDC.BO,SBIN.NS
AXISBANK.BO,0.000313,0.000051,0.000045,0.000082,0.000153
IOC.BO,0.000051,0.002062,-0.000006,0.000056,0.000058
KRITIIND.BO,0.000045,-0.000006,0.001180,0.000117,0.000066
NMDC.BO,0.000082,0.000056,0.000117,0.000372,0.000123
SBIN.NS,0.000153,0.000058,0.000066,0.000123,0.000442


### Max Sharpe Ratio

In [9]:
srs = []
portfolio_stds = []
rand_wts = []
portfolio_returns = []
risk_free_rate = 0
for i in range(0, 20000):
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    rand_wts.append(random_weights)
    # portolfio return
    portfolio_return = np.sum(mean_daily_returns * random_weights)*252
    portfolio_returns.append(portfolio_return)
    # portfolio volatility
    portfolio_std = np.sqrt(np.dot(random_weights.T, np.dot(cov, random_weights))) * np.sqrt(252)
    portfolio_stds.append(portfolio_std)
    # sharpe ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    srs.append(sharpe_ratio)
max_index = srs.index(max(srs))
best_wts = rand_wts[max_index]
max_sr = srs[max_index]
portfolio_sd = portfolio_stds[max_index]
max_return = portfolio_returns[max_index]
print(f"Max Sharpe Ratio: {max_sr}")
print(f"Best Weights: {best_wts.T}")
print(f"Portfolio Volatility: {portfolio_sd*100}%, Sum of weights: {np.sum(best_wts)}")
print(f"Max Expected Annual Return: {max_return*100}%")

Max Sharpe Ratio: [[1.10137345]]
Best Weights: [[0.52032484 0.18440041 0.02873615 0.04269286 0.22384574]]
Portfolio Volatility: [[24.49883702]]%, Sum of weights: 1.0
Max Expected Annual Return: 26.98236865455188%


### Minimum Variance

In [10]:
returns.std()

AXISBANK.BO    0.017697
IOC.BO         0.045414
KRITIIND.BO    0.034347
NMDC.BO        0.019282
SBIN.NS        0.021021
dtype: float64

In [11]:
stds = np.array(returns.std()).reshape(-1, 1)
stds.shape

(5, 1)

In [12]:
product_std = np.dot(stds, stds.T)
product_std

array([[0.00031318, 0.00080368, 0.00060783, 0.00034124, 0.000372  ],
       [0.00080368, 0.00206243, 0.00155984, 0.00087569, 0.00095465],
       [0.00060783, 0.00155984, 0.00117972, 0.00066229, 0.00072201],
       [0.00034124, 0.00087569, 0.00066229, 0.00037181, 0.00040534],
       [0.000372  , 0.00095465, 0.00072201, 0.00040534, 0.00044188]])

In [13]:
cov_mat = np.array(cov)
cov_mat

array([[ 3.13176898e-04,  5.05175906e-05,  4.46357365e-05,
         8.24726570e-05,  1.53254676e-04],
       [ 5.05175906e-05,  2.06243079e-03, -5.83668556e-06,
         5.59780218e-05,  5.77919986e-05],
       [ 4.46357365e-05, -5.83668556e-06,  1.17971839e-03,
         1.16677989e-04,  6.57182327e-05],
       [ 8.24726570e-05,  5.59780218e-05,  1.16677989e-04,
         3.71812827e-04,  1.22674048e-04],
       [ 1.53254676e-04,  5.77919986e-05,  6.57182327e-05,
         1.22674048e-04,  4.41880725e-04]])

#### Correlation Matrix

In [14]:
corr = cov_mat / product_std
corr

array([[ 1.        ,  0.06285764,  0.07343425,  0.24168681,  0.41197072],
       [ 0.06285764,  1.        , -0.00374186,  0.06392425,  0.06053764],
       [ 0.07343425, -0.00374186,  1.        ,  0.17617228,  0.09102154],
       [ 0.24168681,  0.06392425,  0.17617228,  1.        ,  0.30264812],
       [ 0.41197072,  0.06053764,  0.09102154,  0.30264812,  1.        ]])

In [15]:
port_vars = []
rand_wts = []
for i in range(0, 10000):
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    rand_wts.append(random_weights)
    random_weighted_sd = stds * random_weights
    portfolio_var = np.sqrt(np.sum(np.dot(random_weighted_sd.T, np.dot(corr, random_weighted_sd))))*100
    port_vars.append(portfolio_var)
min_index = port_vars.index(min(port_vars))
best_wts = rand_wts[min_index]
min_var = port_vars[min_index]
portfolio_return = np.sum(mean_daily_returns * best_wts)*252
print(f"Best Weights: {best_wts}, Minimum Portfolio Variance: {min_var}, Sum of weights: {np.sum(best_wts)}")
print(f"Return: {portfolio_return*100}%")

Best Weights: [[0.36510317]
 [0.06766384]
 [0.12202986]
 [0.28916003]
 [0.1560431 ]], Minimum Portfolio Variance: 1.3307006605009313, Sum of weights: 1.0000000000000002
Return: 19.939164601395106%


### Monthly, Quarterly, Half-Yearly, Yearly

In [16]:
e = stock_data
monthly = 0
quarterly = 0
half_yearly = 0
yearly = 0
# monthly
start = "2016-04-01"
for i in range(36):
    end = start[:5]
    month = int(start[5:7]) + 1
    if month <= 9:
        end += '0' + str(month) + start[7:]
    elif month > 12:
        end = str(int(start[:4]) + 1) + '-01-01'
    else:
        end += str(month) + start[7:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    monthly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end
# quarterly
start = "2016-04-01"
for i in range(12):
    end = start[:5]
    month = int(start[5:7]) + 3
    if month <= 9:
        end += '0' + str(month) + start[7:]
    elif month == 13:
        end = str(int(start[:4]) + 1) + '-01-01'
    else:
        end += str(month) + start[7:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    quarterly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end
# half-yearly
start = "2016-04-01"
for i in range(6):
    end = start[:5]
    month = int(start[5:7]) + 6
    if month == 16:
        end = str(int(start[:4]) + 1) + '-04-01'
    else:
        end += str(month) + start[7:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    half_yearly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end
# yearly
start = "2016-04-01"
for i in range(3):
    end = str(int(start[:4]) + 1) + start[4:]
    sliced_data = e.loc[(e.index >= start) & (e.index <= end)]
    yearly += (sliced_data.iloc[-1]/sliced_data.iloc[0] - 1)
    start = end

avgs = [list(monthly*100/36), list(quarterly*100/12), list(half_yearly*100/6), list(yearly*100/3)]
print(f"Monthly Avg Returns: {avgs[0]}")
print(f"Quarterly Avg Return: {avgs[1]}")
print(f"Half Yearly Avg Return: {avgs[2]}")
print(f"Yearly Avg Return: {avgs[3]}")

Monthly Avg Returns: [1.8588962544346903, 2.5834192184126907, 0.41000984388702855, 0.5522924367301297, 1.9931008006740771]
Quarterly Avg Return: [5.443626709232184, 7.440080264969452, 0.3237652138705989, 1.5293753691705188, 5.3116615893380015]
Half Yearly Avg Return: [10.750451815842801, 16.17892489825639, 0.12905901826752256, 3.379752258880238, 9.931464358435624]
Yearly Avg Return: [23.309802871503535, 42.891757557849104, 2.7478150057574258, 8.933614280521, 22.514846391858487]


In [17]:
for avg in avgs:
    random_weights = np.random.dirichlet(np.ones(5), size = 1).T
    total_return = avg * random_weights.T
    print(avg)
    print(random_weights.T)
    print(total_return)
    print(np.sum(total_return))

[1.8588962544346903, 2.5834192184126907, 0.41000984388702855, 0.5522924367301297, 1.9931008006740771]
[[0.17590559 0.09856396 0.19498086 0.49692799 0.0336216 ]]
[[0.32699024 0.25463202 0.07994407 0.27444957 0.06701125]]
1.0030271510032864
[5.443626709232184, 7.440080264969452, 0.3237652138705989, 1.5293753691705188, 5.3116615893380015]
[[0.19597723 0.31692101 0.10368083 0.24573446 0.13768646]]
[[1.06682689 2.35791778 0.03356825 0.37582023 0.7313439 ]]
4.565477038923461
[10.750451815842801, 16.17892489825639, 0.12905901826752256, 3.379752258880238, 9.931464358435624]
[[0.050236   0.21080162 0.26986886 0.19308995 0.27600357]]
[[0.54005975 3.41054355 0.03482901 0.6525962  2.74111959]]
7.379148096809535
[23.309802871503535, 42.891757557849104, 2.7478150057574258, 8.933614280521, 22.514846391858487]
[[0.14094787 0.2891806  0.25002137 0.04966862 0.27018154]]
[[ 3.28546701 12.40346413  0.68701247  0.44372031  6.08309594]]
22.90275986402436


In [18]:
types = ['Monthly', 'Quarterly', 'Half-Yearly', 'Yearly']
best_wts_for_avgs = []
annual_returns_for_avgs = []
port_vars_for_avgs = []
for avg in avgs:
    min_port_var = 1.42
    max_port_var = 1.47
    port_vars = []
    returns = []
    rand_wts = []
    for i in range(0, 10000):
        random_weights = np.random.dirichlet(np.ones(5), size = 1).T
        random_weighted_sd = stds * random_weights
        portfolio_var = np.sqrt(np.sum(np.dot(random_weighted_sd.T, np.dot(corr, random_weighted_sd))))*100
        if (portfolio_var >= min_port_var) & (portfolio_var <= max_port_var):
            port_vars.append(portfolio_var)
            rand_wts.append(random_weights)
            total_return = np.sum(avg * random_weights.T)
            returns.append(total_return)
    max_index = returns.index(max(returns))
    max_return = returns[max_index]
    best_wts = rand_wts[max_index]
    min_var = port_vars[max_index]

    best_wts_for_avgs.append(best_wts)
    port_vars_for_avgs.append(min_var)
    annual_ret = np.sum(mean_daily_returns * best_wts)*252
    annual_returns_for_avgs.append(annual_ret)

max_index = annual_returns_for_avgs.index(max(annual_returns_for_avgs))
maximum_return = annual_returns_for_avgs[max_index]
bestest_wts = best_wts_for_avgs[max_index]
miniest_var = port_vars_for_avgs[max_index]
print(f"Best Performance: {types[max_index]}")
print(f"Best Return: {maximum_return*100}%, Minimum Portfolio Variance: {miniest_var}")
print(f"Best Weights: {bestest_wts}, Sum of weights: {np.sum(bestest_wts)}")

Best Performance: Yearly
Best Return: 25.330269687385375%, Minimum Portfolio Variance: 1.468733802514229
Best Weights: [[0.41873537]
 [0.14500007]
 [0.02931653]
 [0.09309007]
 [0.31385797]], Sum of weights: 1.0


In [19]:
ordered_types = {}
for i in range(4):
    ordered_types[types[i]] = annual_returns_for_avgs[i]*100
print(ordered_types)

{'Monthly': 24.45750341827336, 'Quarterly': 24.36966589915857, 'Half-Yearly': 25.10544299353945, 'Yearly': 25.330269687385375}
